# Estudo sobre a base do jogo Jokenpower

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('attacks_pattern.csv')
df

,timestamp,time_since_start,player_type,attack_type,damage,is_counter_attack
0,2022-09-14 23:26:43.497102,3601,Main,Paper,1,False
1,2022-09-14 23:26:44.379765,4482,Enemy,Paper,1,False
2,2022-09-14 23:26:45.057001,5160,Main,Paper,1,False
3,2022-09-14 23:26:46.638571,6742,Main,Paper,1,False
4,2022-09-14 23:26:47.604161,7708,Enemy,Rock,10,False
...,...,...,...,...,...,...
96,2022-09-16 16:39:37.921929,13714,Enemy,Scissors,3,False
97,2022-09-16 16:39:38.789459,14581,Main,Paper,1,False
98,2022-09-16 16:39:39.718474,15511,Enemy,Paper,1,False
99,2022-09-16 16:39:40.876221,16668,Main,Rock,10,False


## Tratamento de dados

In [2]:
# Parsear string timestamp para Tipo Timestamp do pandas
df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)

In [3]:
# Criar campo com diferença de tempo entre ataques
df['time_diff'] = df['timestamp'].diff().dt.total_seconds()

In [4]:
# Definir início de partida: quando próximo time_since_start for menor que anterior
df.loc[df['time_since_start'].diff() < 0, 'time_diff'] = np.nan

In [5]:
# Deletar NaN da tabela
df.dropna(inplace=True)

In [6]:
# Normalizando diferença de tempo entre ataques
df['label_time_diff'] = df['time_diff'].apply(lambda x: int(x*10))

In [7]:
# Normalizando valores qualitativos para quantitativos

def get_norm_attack_type(atk: str):
    result = 99
    if atk == 'Rock':
        result = 0
    elif atk == 'Paper':
        result = 1
    elif atk == 'Scissors':
        result = 2
    return result

df['label_attack_type'] = df['attack_type'].apply(get_norm_attack_type)

In [8]:
df

,timestamp,time_since_start,player_type,attack_type,damage,is_counter_attack,time_diff,label_time_diff,label_attack_type
1,2022-09-14 23:26:44.379765,4482,Enemy,Paper,1,False,0.882663,8,1
2,2022-09-14 23:26:45.057001,5160,Main,Paper,1,False,0.677236,6,1
3,2022-09-14 23:26:46.638571,6742,Main,Paper,1,False,1.581570,15,1
4,2022-09-14 23:26:47.604161,7708,Enemy,Rock,10,False,0.965590,9,0
5,2022-09-14 23:26:48.214027,8317,Main,Scissors,3,False,0.609866,6,2
...,...,...,...,...,...,...,...,...,...
95,2022-09-16 16:39:37.003940,12796,Main,Scissors,3,False,1.179497,11,2
96,2022-09-16 16:39:37.921929,13714,Enemy,Scissors,3,False,0.917989,9,2
97,2022-09-16 16:39:38.789459,14581,Main,Paper,1,False,0.867530,8,1
98,2022-09-16 16:39:39.718474,15511,Enemy,Paper,1,False,0.929015,9,1


## Scikit Learn

In [9]:
# Pegar apenas danos ao inimigo = Apenas do player
df_only_player = df.loc[df['damaged_player'] == 'Enemy']

In [10]:
# Filtrar apenas colunas necessárias para fator X
x_chance_contra_ataque = df_only_player.loc[:, ['label_attack_type', 'label_time_diff']].values
x_chance_contra_ataque

array([[ 1,  8],
       [ 0,  9],
       [ 1, 15],
       [ 1,  7],
       [ 2,  8],
       [ 1, 16],
       [ 1,  7],
       [ 2, 11],
       [ 0, 14],
       [ 1,  9],
       [ 2, 12],
       [ 2,  7],
       [ 2, 15],
       [ 2,  7],
       [ 2,  4],
       [ 1, 16],
       [ 1,  7],
       [ 1,  6],
       [ 0,  6],
       [ 1, 13],
       [ 1,  7],
       [ 1,  3],
       [ 0, 15],
       [ 1,  8],
       [ 0, 10],
       [ 0, 23],
       [ 0, 10],
       [ 0, 12],
       [ 1, 14],
       [ 0,  5],
       [ 2,  9],
       [ 1,  9]])

In [11]:
# Filtrar apenas coluna necessária para fator Y
y_chance_contra_ataque = df_only_player.loc[:, 'is_counter_attack'].values
y_chance_contra_ataque

array([False, False,  True, False, False,  True, False, False, False,
       False,  True, False,  True, False, False,  True, False, False,
       False,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False])

### Normalização de valores qualitativos para quantitativos

In [12]:
# Normalização deve ser feita para o modelo GaussianNB aceitar como input

# from sklearn.preprocessing import LabelEncoder
# label_encoder_attack_type = LabelEncoder()
# x_chance_contra_ataque[:,0] = label_encoder_attack_type.fit_transform(x_chance_contra_ataque[:,0])

### Treinamento do modelo com algoritmo Naive Bayes

In [13]:
from sklearn.naive_bayes import GaussianNB

In [14]:
naive_chance_contra_ataque = GaussianNB()
naive_chance_contra_ataque.fit(x_chance_contra_ataque, y_chance_contra_ataque)

GaussianNB()

### Previsão

In [15]:
previsao = naive_chance_contra_ataque.predict([[0, 7]])
previsao

array([False])